 # Bharath Gunasekaran
 CMPE 258 HW 6 Part 2a

 Creating a CNN in Pytorch

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable


# Loading Dataset

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [3]:
trainset = torchvision.datasets.CIFAR10(root="data/train",
    train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root="data/test",
    train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=2)


Extracting data/train/cifar-10-python.tar.gz to data/train



Extracting data/test/cifar-10-python.tar.gz to data/test


# Creating Model

In [4]:
import torch
import torch.nn as nn


class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

# Training the model

In [5]:
model = CNN()
CUDA=torch.cuda.is_available()
if CUDA:
  model=model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

In [7]:

for epoch in range(0, 10):

    running_loss = 0.0
    running_accuracy = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data


        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        CUDA=torch.cuda.is_available()
        if CUDA:
          inputs=inputs.cuda()
          labels=labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()


        # print statistics
        running_loss += loss.item()
        running_accuracy += accuracy(outputs,labels)

    # Normalizing the loss by the total number of train batches
    running_loss /= len(trainloader)
    running_accuracy /= len(trainloader)

    print("Iteration: {0} | Loss: {1} | Training accuracy: {2}".format(epoch+1, running_loss, running_accuracy))

print('==> Finished Training ...')

Iteration: 1 | Loss: 1.61940789588577 | Training accuracy: 0.386413037776947
Iteration: 2 | Loss: 1.180261031288625 | Training accuracy: 0.5691216588020325
Iteration: 3 | Loss: 1.000452625324659 | Training accuracy: 0.6432984471321106
Iteration: 4 | Loss: 0.8844977853548192 | Training accuracy: 0.6880714893341064
Iteration: 5 | Loss: 0.7968880921373587 | Training accuracy: 0.7221107482910156
Iteration: 6 | Loss: 0.7361554978296275 | Training accuracy: 0.7435302138328552
Iteration: 7 | Loss: 0.6889971762971805 | Training accuracy: 0.7606337666511536
Iteration: 8 | Loss: 0.6442366682964823 | Training accuracy: 0.7787763476371765
Iteration: 9 | Loss: 0.610231879422122 | Training accuracy: 0.7903333306312561
Iteration: 10 | Loss: 0.5849387026808756 | Training accuracy: 0.7983015775680542
==> Finished Training ...
